# Required libraries

In [1]:
# import Pkg; Pkg.add("CSV")
using CSV

# import Pkg; Pkg.add("Dates")
using Dates

# import Pkg; Pkg.add("DataFrames")
using DataFrames

# Print Date

In [2]:
function updated() 
    println( "Last update: ", Dates.format(Dates.now(), "u. d, yyyy HH:mm"))
end

updated (generic function with 1 method)

# Declare known data files

In [3]:
file_ISM_Mfc_Emp = "data/united-states.ism-manufacturing-employment.csv"
file_Markkit_Mfc_PMI = "data/united-states.markit-manufacturing-pmi.csv"

"data/united-states.markit-manufacturing-pmi.csv"

# List available functions

In [4]:
println("Functions available:")
println("df = fetch_data(file_path, date_format=\"yyyy-mm-dd\" )")
println("df = shift_dates_by_days!(df, shift_days=-1, date_column=1)")

Functions available:
df = fetch_data(file_path, date_format="yyyy-mm-dd" )
df = shift_dates_by_days!(df, shift_days=-1, date_column=1)


# Define Fetch Data function¶

In the future, I expect to have large data sets that are too big for the GitHub, so I am planning for the separate downlod location i.e. Google Drive, etc.

In [5]:
function fetch_data(file_path, date_format="yyyy-mm-dd")
    isfile(file_path) || # does file exist locally?
        download( string("https://raw.githubusercontent.com/UkiDLucas/MarketIndicators.jl/master/", file_path), # URL
        file_path) # save it as name
    

    
    return CSV.read(        # returns DataFrame
        file_path, 
        dateformat="$date_format"
        )
end

fetch_data (generic function with 2 methods)

In [6]:
# TEST
#df = fetch_data(file_ISM_Mfc_Emp, "yyyy.mm.dd")

# Shift DataFrame Date by x days

In [7]:
function shift_dates_by_days!(df::DataFrame, shift_days::Int64=-1, date_column::Int64=1)
    rows = size(df)[1]
    
    for row in 1:rows
        original = Date(df[row, date_column]) 
        shifted = original + Dates.Day(shift_days)
        # println("original ", original, " ", shift_days, " shift ", shifted)
        df[row, date_column] = shifted
    end
    return df
end

# df = shift_dates_by_days!(df3, -90)

shift_dates_by_days! (generic function with 3 methods)

# Rata Die

In [8]:
first_date = Date("Jan. 1, 1900", "u. dd, yyyy")

1900-01-01

In [9]:
day_one = Dates.datetime2rata(first_date) # typeof(rate_die) = Int64

693596

In [10]:
function days_since_day_one(df, row, date_column) 
    date = df[row, date_column] # type Date
    rata_die = Dates.datetime2rata(date) 
    rata_die = rata_die - day_one
    return rata_die
end

days_since_day_one (generic function with 1 method)

In [11]:
function update_rata_die!(df, days_column=1, date_column=2)
    rows = size(df)[1] # first part of the returned tupple

    for row = 1:rows
        days = days_since_day_one(df, row, date_column)
        # println(days)
        df[row, days_column] = days
    end
    return df
end

update_rata_die! (generic function with 3 methods)

In [12]:
function insert_rata_die_column!(df)
    rows = size(df)[1] # first number of the tuple
    # https://stackoverflow.com/a/63731422/6312771
    insertcols!(df, 1,  :Day => 1:rows, makeunique=true) # insert as column 1, populate with 1,2,3,..
end

insert_rata_die_column! (generic function with 1 method)